# Select Worksheet

Provide an example for the selection of a worksheet using plotly, and how the previous worksheet score modifies the target score for the model.

## Setup

In [17]:
import plotly.graph_objs as go
import plotly.express as px
import numpy as np

## Target score based on previous score

In [18]:
def target_score(prev_score, m):
    """
    Calculates the target score as a percentage based on the previous score and the slope coefficient.

    Parameters:
        prev_score (float): The previous score as a percentage.
        m (float): The slope coefficient that determines the slope of the line.

    Returns:
        float: The target score as a percentage based on the previous score and the slope coefficient.

    Example:
        >>> target_score(50, -0.75)
        62.5
    """
    return m * prev_score + 100

x = np.linspace(0, 100, 100)
y = target_score(x, m=-0.75)

In [19]:
# Create the initial plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))
fig.update_layout(template="plotly_dark", title="Target Score Based on Previous Score")
# fix the y axis range
fig.update_yaxes(range=[0, 100])

# Define the slider
slider = {
    'min':0,
    'max':-1,
    'step':-0.01,
    'value':-0.5
    }

# Get the index of the step that corresponds to the value of -0.75
initial_step_index = int((slider['max'] - slider['value']) / slider['step'])

# Define the layout
layout = dict(
    xaxis=dict(title='Previous Score %', 
              title_font=dict(size=18, family='Monaco,monospace')),
    yaxis=dict(title='Target Score %', 
              title_font=dict(size=18, family='Monaco,monospace')),
    font=dict(family='Monaco,monospace', size=12),
    sliders=[dict(
        active=initial_step_index,
        tickwidth=2,
        transition=dict(duration=300, easing='cubic-in-out'),
        pad=dict(t=10),
        len=0.9,
        x=0.1,
        y=-0.2,
        currentvalue=dict(visible=True, 
                          prefix='Slope Parameter : ', 
                          xanchor='left', 
                          font=dict(size=16, color='orange')),  # Set the initial value of the slider to -0.75
        steps=[dict(
            label='{:.2f}'.format(m),
            method='update',
            args=[{'y': [target_score(x, m)]}]
        ) for m in np.arange(slider['min'], slider['max']+slider['step'], slider['step'])]
    )]
)

# Update the layout
fig.update_layout(layout)

# Show the plot
fig.show()

In [20]:
fig.write_json("../content/projects/edplace/target_score.json")

## Worksheet distance to target

In [21]:
def distance_to_target(score, target_score):
    """
    Calculates the distance to the target score as a percentage.

    Parameters:
        score (float or list of floats): The current score(s) as a percentage.
        target_score (float): The target score as a percentage.

    Returns:
        float or list of floats: The distance(s) to the target score as a percentage.

    Raises:
        ValueError: If any score is outside the range of 0 to 100.
        ValueError: If the target score is outside the range of 0 to 100.

    Example:
        >>> distance_to_target(75, 80)
        75.0
        >>> distance_to_target([70, 80, 90], 85)
        [15.0, 5.0, 5.0]
    """
    if np.isscalar(score) == False:
        assert all(0 <= s <= 100 for s in score), "Score must be between 0 and 100."
        return [100 - abs(s - target_score) for s in score]
    else:
        assert 0 <= score <= 100, "Score must be between 0 and 100."
        assert 0 <= target_score <= 100, "Target score must be between 0 and 100."
        return 100 - abs(score - target_score)


x = np.linspace(0, 100, 100)
y = distance_to_target(x, 75)

px.line(x=x, y=y, 
        template="plotly_dark", 
        title="Target Score Based on Previous Score").update_yaxes(range=[0, 100]).update_xaxes(range=[0, 100])

In [22]:
# Create the initial plot
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='lines'))
fig.update_layout(template="plotly_dark", title="Worksheet Score Distance to Target")
# fix the y axis range
fig.update_yaxes(range=[0, 100]).update_xaxes(range=[0, 100])

# Define the slider
slider = {
    'min':0,
    'max':100,
    'step':1,
    'value':75
    }

# Get the index of the step that corresponds to the value of -0.75
initial_step_index = int(slider['value'] / slider['step'])

# Define the layout
layout = dict(
    xaxis=dict(title='Average Score on Worksheet %', 
              title_font=dict(size=18, family='Monaco,monospace')),
    yaxis=dict(title='Proximity to Target %', 
              title_font=dict(size=18, family='Monaco,monospace')),
    font=dict(family='Monaco,monospace', size=12),
    sliders=[dict(
        active=initial_step_index,
        tickwidth=2,
        transition=dict(duration=300, easing='cubic-in-out'),
        pad=dict(t=10),
        len=0.9,
        x=0.1,
        y=-0.2,
        currentvalue=dict(visible=True, 
                          prefix='Target Score : ',
                          suffix='%', 
                          xanchor='left', 
                          font=dict(size=16, color='orange')),  # Set the initial value of the slider to -0.75
        steps=[dict(
            label='{:.0f}'.format(target_score),
            method='update',
            args=[{'y': [distance_to_target(x, target_score)]}]
        ) for target_score in np.arange(slider['min'], slider['max']+slider['step'], slider['step'])]
    )]
)

# Update the layout
fig.update_layout(layout)

# Show the plot
fig.show()

In [23]:
fig.write_json("../content/projects/edplace/proximity_to_target.json")